# HydroSOS - Persistence and Hydrological Analogues method

In this notebook, we explore the Persistance and Hydrological Analogue method

## Import the python libraries

In [42]:
%reset -f

In [43]:
# Importing the libraries
import pandas as pd
import seaborn as sns
import numpy as np
import calendar
import datetime
import matplotlib.pyplot as plt
import math
from scipy import stats
plt.style.use('classic')
%matplotlib inline

from IPython.display import HTML

sns.set()

## Functions

In [44]:
# Quantile Function Definitions
def q1(x):
    return x.quantile(0.28)

def q2(x):
    return x.median()

def q3(x):
    return x.quantile(0.72)

def q5(x):
    return x.quantile(0.05)

def q95(x):
    return x.quantile(0.95)

def q87(x):
    return x.quantile(0.87)

def q13(x):
    return x.quantile(0.13)


## Import Data 

In this example we import the daily discharge data

In [45]:
today = datetime.date.today()

In [46]:
today

datetime.date(2023, 4, 28)

In [47]:
DISCHARGE_DAILY = pd.read_csv('../data/santalucia_caudales_27abr2023.csv',parse_dates=['Fecha'],index_col="Fecha",dayfirst=True,na_values="NA")
# Identify the missing data
DISCHARGE_DAILY_date_missing = pd.date_range(start = '1980-01-01', end = today ,freq='D')
DISCHARGE_DAILY = DISCHARGE_DAILY.reindex(DISCHARGE_DAILY_date_missing,fill_value=None)
DISCHARGE_DAILY.index.name = 'Fecha' # Set index Fecha
DISCHARGE_DAILY.columns = ['Station','Discharge']
DISCHARGE_DAILY = DISCHARGE_DAILY.drop(columns='Station')
HTML(DISCHARGE_DAILY.head(6).to_html())

,Discharge
Fecha,
1980-01-01,1.738
1980-01-02,1.738
1980-01-03,1.738
1980-01-04,1.738
1980-01-05,1.738
1980-01-06,1.738


## Calculate monthly mean from daily data

First, we identify the months which contains lower that 5 null value observation.

In [48]:
# group the Dataframe in a monthly time scale
GROUPER_DISCHARGE_MONTHLY = DISCHARGE_DAILY.groupby(pd.Grouper(freq='1MS'))
NUMBER_MISSING = GROUPER_DISCHARGE_MONTHLY.apply(lambda x: pd.isnull(x).sum()).unstack(1)
# sum the number of null values and assing a "true" if the number of null values are lower that a number for a given month
BOOL_MISSING = GROUPER_DISCHARGE_MONTHLY.apply(lambda x: pd.isnull(x).sum()).unstack(1) < 5
BOOL_MISSING = BOOL_MISSING.to_frame() # convert to DataFrame
BOOL_MISSING.columns = ['missing'] # change the column name to "missing"

NUMBER_MISSING = NUMBER_MISSING.to_frame()
NUMBER_MISSING.columns = ['number_missing'] # change the column name to "missing"

After this, we calculate the monthly mean flow only if the null values in that month are below 5 values. Also define de water year starting in april.

In [53]:
DISCHARGE_MONTHLY = DISCHARGE_DAILY.resample('M').apply(lambda x: x.mean() if x.isnull().sum() < 10 else np.nan)
DISCHARGE_MONTHLY['Year'] = DISCHARGE_MONTHLY.index.year
DISCHARGE_MONTHLY['Month'] = DISCHARGE_MONTHLY.index.month
DISCHARGE_MONTHLY['water_year'] = DISCHARGE_MONTHLY.index.year.where(DISCHARGE_MONTHLY.index.month < 4, DISCHARGE_MONTHLY.index.year + 1)

HTML(DISCHARGE_MONTHLY.head(18).to_html(index=False))

Discharge,Year,Month,water_year
1.198613,1980,1,1980
28.386414,1980,2,1980
86.782323,1980,3,1980
172.187367,1980,4,1981
275.560258,1980,5,1981
228.526533,1980,6,1981
203.495161,1980,7,1981
167.313355,1980,8,1981
61.411033,1980,9,1981
138.581516,1980,10,1981


In [54]:
## create column for day, month, year in the daily discharge ()
# DISCHARGE_DAILY['Year'] = DISCHARGE_DAILY.index.year
# DISCHARGE_DAILY['Month'] = DISCHARGE_DAILY.index.month
# DISCHARGE_DAILY['Day'] = DISCHARGE_DAILY.index.day
# DISCHARGE_DAILY['monthday'] = DISCHARGE_DAILY.index.day_of_year
# DISCHARGE_DAILY['water_year'] = DISCHARGE_DAILY.index.year.where(DISCHARGE_DAILY.index.month < 4, DISCHARGE_DAILY.index.year + 1)
## 
# DISCHARGE_MONTHLY.to_clipboard()

## Select by Climatology range

First, we transform the discharge to a logaritmic scale and we insert it in the dataframe

In [55]:
DISCHARGE_MONTHLY['Q_to_log'] = np.log(DISCHARGE_MONTHLY['Discharge'])

In this part we select the reference period for analysis. In this case we use the 1980/01-2023/01 period

In [58]:
DISCHARGE_MONTHLY_SELECTED = DISCHARGE_MONTHLY.loc['1980-01-01': '2023-05-01']
HTML(DISCHARGE_MONTHLY_SELECTED.tail(6).to_html(index=False))

Discharge,Year,Month,water_year,Q_to_log
11.738167,2022,11,2023,2.462846
5.969037,2022,12,2023,1.786586
5.419419,2023,1,2023,1.689989
5.557464,2023,2,2023,1.715142
3.336120,2023,3,2023,1.204808
2.693037,2023,4,2024,0.990670


Extract the years from the monthly discharge dataframe

In [59]:
YEAR_SELECTED = np.unique(DISCHARGE_MONTHLY_SELECTED['Year'])
display(YEAR_SELECTED)

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023],
      dtype=int64)

In [60]:
len(YEAR_SELECTED)

44

## Calculate Monthly statistics for Q Log-transformed 

In this part we calculate monthly statistcs for the QLog Variable.

 The statistis include: mean, min, max, std, q1(quantile 0.28), q3(quantile 0.72), q5(0.05), q95(0.95), q87(0.87), q13(0.13)

In [61]:
DISCHARGE_MONTHLY_STATS = DISCHARGE_MONTHLY_SELECTED.Q_to_log.groupby(DISCHARGE_MONTHLY_SELECTED.index.month).agg([np.min, q1, q2, np.mean, q3, np.max, np.std, q95, q5, q87, q13])
# Display results
HTML(DISCHARGE_MONTHLY_STATS.to_html())

,amin,q1,q2,mean,q3,amax,std,q95,q5,q87,q13
Fecha,,,,,,,,,,,
1,0.181165,1.914800,2.390056,2.620895,3.030727,5.409375,1.148784,4.550919,1.134860,3.927229,1.691085
2,1.178985,2.084481,2.580642,2.938515,3.278563,6.574466,1.285527,6.006143,1.487074,4.052257,1.793204
3,1.204808,2.338748,2.653676,3.096355,3.665468,6.178586,1.219652,5.221666,1.490676,4.534554,1.893588
4,0.990670,2.459688,3.023710,3.405855,4.257166,6.479732,1.451209,6.254593,1.671804,5.142693,1.986086
5,1.418020,2.812112,3.792547,3.826760,4.756044,6.520439,1.304600,6.061842,2.152025,5.395068,2.495237
6,1.814417,3.716403,4.773276,4.431915,5.194240,6.586199,1.255580,6.085650,2.257051,5.876834,2.886427
7,2.190658,4.272702,4.828448,4.646775,5.252465,6.317149,0.998952,6.115233,2.839401,5.694232,3.305521
8,1.736520,3.996108,4.524422,4.529553,5.138838,6.566886,1.008155,5.926632,2.991829,5.482504,3.463987
9,2.669799,4.015859,4.846679,4.584747,5.099869,6.277342,0.969279,6.129177,3.063143,5.521344,3.408601


In [62]:
## Calculate Daily statistics

#We reproduce the same procedure for the Daily dataframe

# DISCHARGE_DAILY_STATS = DISCHARGE_DAILY.Discharge.groupby(DISCHARGE_DAILY.monthday).agg([np.min, q1, q2, np.mean, q3, np.max, np.std, q5, q95, q87, q13])
# HTML(DISCHARGE_DAILY_STATS.head(12).to_html())

We re-shape the monthly log discharge dataframe using a pivot table for the Discharge (pivQ) and the log Discharge (pivLogQ)

In [63]:
DISCHARGE_MONTHLY_PIVOT = pd.pivot_table(DISCHARGE_MONTHLY_SELECTED, index=['Month'],columns=['Year'], values=['Discharge'],dropna=False)

In [64]:
DISCHARGE_MONTHLY_PIVOT.columns = YEAR_SELECTED
HTML(DISCHARGE_MONTHLY_PIVOT.to_html())

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.198613,9.649097,2.314161,5.504935,31.699452,3.292613,6.365613,7.701710,35.197677,3.159548,3.394032,9.386581,11.547161,8.764806,61.059097,5.666419,5.652871,6.823387,86.377258,139.122200,5.794742,NaN,16.689323,92.822161,17.708484,17.594226,223.491774,8.624258,12.686935,10.022548,47.671871,11.289742,NaN,39.033710,NaN,17.394710,35.030613,17.318323,19.959806,NaN,8.826548,10.550968,51.006968,5.419419
2,28.386414,36.472929,25.551107,21.500929,442.865069,3.742321,11.727821,6.983821,24.039759,3.251071,46.210500,6.463036,10.571966,168.635464,NaN,20.942964,4.507172,7.340786,9.490786,402.004357,4.695931,NaN,18.777107,64.750000,11.498552,35.593357,15.826429,7.956071,12.782207,24.839643,716.562679,11.308107,NaN,13.205607,NaN,11.374393,6.038690,39.699107,8.088250,NaN,7.272966,57.094500,13.658571,5.557464
3,86.782323,14.206161,13.985161,6.928129,94.854258,54.831200,7.393065,36.113290,185.242548,6.573774,34.673032,10.401032,10.360065,13.017355,27.706136,21.365710,4.440097,9.754258,8.118677,146.427452,4.304742,137.535452,482.309581,28.741935,10.539452,15.743710,24.816452,380.226839,58.918226,39.851903,43.222032,11.491323,NaN,NaN,NaN,9.679593,5.805742,11.365097,25.362968,12.775387,6.074484,52.614710,13.748516,3.336120
4,172.187367,23.795433,18.486100,17.752067,23.866967,70.309300,13.195200,13.115286,82.727500,7.237767,376.740567,9.159867,25.703900,118.242067,26.678067,158.291933,68.899367,5.659167,21.786667,74.156100,NaN,19.416467,615.286133,11.077067,25.058567,349.290667,14.848700,515.830000,11.676167,11.703333,57.596767,11.241733,12.866733,NaN,NaN,NaN,651.796200,7.382300,7.279833,4.442733,5.372567,79.801567,12.372433,2.693037
5,275.560258,483.646645,154.619290,17.618194,117.863032,127.480194,69.309516,12.845387,16.410258,4.128935,49.336677,44.369258,148.086903,154.365581,203.886774,59.106774,14.019452,9.236452,25.933742,20.219871,678.876323,38.408742,306.391677,110.181548,70.098258,224.627613,12.679355,429.165290,14.860677,12.005419,30.820226,12.613290,9.339032,NaN,NaN,8.602258,48.998742,52.842226,22.766839,49.218903,5.638161,40.936258,36.368806,NaN
6,228.526533,68.479367,368.022200,120.220567,360.682667,199.745367,159.207600,9.757933,17.794967,6.137500,105.698333,132.545633,161.069200,118.306200,38.216333,124.082800,9.554867,64.082167,56.815633,136.550833,158.643400,400.529767,211.609267,328.281933,73.318367,576.498467,439.505467,63.968367,27.394167,40.479867,181.485167,NaN,18.476033,19.267654,NaN,8.618967,57.570000,28.894033,43.763333,725.019800,175.301300,341.100267,15.803767,NaN
7,203.495161,144.611613,190.826290,74.770710,503.148516,193.472935,114.704484,100.836000,53.492613,20.860677,20.685516,194.289258,138.602903,66.109742,62.721742,171.670129,11.587871,17.459742,263.793677,553.991097,360.227774,75.633903,160.956065,172.914742,82.276387,142.360774,129.163161,26.138161,72.224452,302.760516,299.823355,NaN,47.711677,NaN,NaN,8.941097,450.191258,123.089613,252.009677,102.288806,61.293645,91.588548,126.974065,NaN
8,167.313355,236.170935,192.769581,346.293065,37.918839,189.799194,711.151839,91.027935,85.292806,89.494355,12.375839,221.893419,45.405548,95.442258,170.738000,22.440097,5.677548,78.653258,53.613774,91.667677,NaN,190.066645,157.920032,149.917645,43.133419,30.432065,129.506000,70.033774,32.681387,64.109032,252.388000,NaN,NaN,NaN,NaN,249.796194,168.002161,514.922194,NaN,102.432742,44.095097,92.242548,25.454323,NaN
9,61.411033,162.816267,142.179767,406.849300,86.823867,141.636433,240.833867,54.982367,31.039967,22.522833,22.500033,191.928767,164.103800,185.151900,222.238267,17.490333,157.591793,29.496267,43.820633,147.385833,NaN,101.799167,134.028767,455.033767,127.316900,65.958733,30.571067,270.554333,37.573000,70.706667,532.371667,NaN,NaN,NaN,NaN,32.942333,191.9241

In [65]:
LOG_DISCHARGE_MONTHLY_PIVOT = pd.pivot_table(DISCHARGE_MONTHLY_SELECTED, index=['Month'],columns=['Year'], values=['Q_to_log'],dropna=False)

Rename the columns based on the reference years

In [66]:
LOG_DISCHARGE_MONTHLY_PIVOT.columns = YEAR_SELECTED
HTML(LOG_DISCHARGE_MONTHLY_PIVOT.to_html())

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.181165,2.266864,0.839047,1.705645,3.456299,1.191681,1.850911,2.041442,3.560980,1.150429,1.222019,2.239281,2.446440,2.170744,4.111842,1.734557,1.732164,1.920356,4.458724,4.935353,1.756951,NaN,2.814769,4.530685,2.874044,2.867571,5.409375,2.154579,2.540573,2.304837,3.864342,2.423895,NaN,3.664426,NaN,2.856166,3.556222,2.851765,2.993721,NaN,2.177764,2.356218,3.931962,1.689989
2,3.345911,3.596570,3.240681,3.068096,6.093265,1.319706,2.461964,1.943596,3.179709,1.178985,3.833207,1.866099,2.358206,5.127739,NaN,3.041803,1.505670,1.993446,2.250321,5.996463,1.546696,NaN,2.932638,4.170534,2.442221,3.572159,2.761681,2.073935,2.548054,3.212441,6.574466,2.425520,NaN,2.580642,NaN,2.431365,1.798187,3.681329,2.090412,NaN,1.984164,4.044708,2.614367,1.715142
3,4.463403,2.653676,2.637997,1.935590,4.552342,4.004259,2.000542,3.586661,5.221666,1.883088,3.545962,2.341905,2.337958,2.566283,3.321654,3.061787,1.490676,2.277704,2.094167,4.986530,1.459717,4.923882,6.178586,3.358357,2.355126,2.756441,3.211507,5.940768,4.076150,3.685170,3.766350,2.441592,NaN,NaN,NaN,2.270020,1.758847,2.430547,3.233290,2.547520,1.804097,3.962996,2.620931,1.204808
4,5.148583,3.169494,2.917019,2.876502,3.172495,4.252904,2.579853,2.573778,4.415552,1.979313,5.931557,2.214832,3.246643,4.772734,3.283842,5.064441,4.232647,1.733277,3.081298,4.306172,NaN,2.966122,6.422087,2.404877,3.221216,5.855904,2.697912,6.245777,2.457550,2.459874,4.053466,2.419633,2.554645,NaN,NaN,NaN,6.479732,1.999085,1.985108,1.491270,1.681306,4.379543,2.515471,0.990670
5,5.618806,6.181355,5.040966,2.868932,4.769523,4.847961,4.238582,2.552985,2.797907,1.418020,3.898668,3.792547,4.997799,5.039324,5.317565,4.079346,2.640446,2.223158,3.255545,3.006666,6.520439,3.648285,5.724864,4.702129,4.249898,5.414444,2.539975,6.061842,2.698719,2.485358,3.428171,2.534751,2.234203,NaN,NaN,2.152025,3.891795,3.967311,3.125305,3.896278,1.729558,3.712016,3.593711,NaN
6,5.431652,4.226532,5.908143,4.789328,5.887999,5.297043,5.070209,2.278081,2.878916,1.814417,4.660589,4.886927,5.081834,4.773276,3.643263,4.820949,2.257051,4.160166,4.039812,4.916697,5.066659,5.992788,5.354741,5.793873,4.294811,6.356973,6.085650,4.158389,3.310330,3.700805,5.201174,NaN,2.916474,2.958428,NaN,2.153965,4.053002,3.363635,3.778796,6.586199,5.166506,5.832176,2.760248,NaN
7,5.315642,4.974052,5.251364,4.314426,6.220885,5.265138,4.742359,4.613495,3.979544,3.037866,3.029434,5.269348,4.931613,4.191316,4.138708,5.145575,2.449959,2.859898,5.575167,6.317149,5.886737,4.325905,5.081131,5.152799,4.410084,4.958364,4.861076,3.263396,4.279779,5.712942,5.703193,NaN,3.865176,NaN,NaN,2.190658,6.109673,4.812913,5.529467,4.627800,4.115676,4.517306,4.843983,NaN
8,5.119868,5.464556,5.261496,5.847285,3.635448,5.245967,6.566886,4.511166,4.446090,4.494176,2.515746,5.402197,3.815634,4.558521,5.140130,3.110849,1.736520,4.365049,3.981806,4.518170,NaN,5.247375,5.062089,5.010086,3.764298,3.415497,4.863727,4.248978,3.486806,4.160585,5.530968,NaN,NaN,NaN,NaN,5.520645,5.123977,6.244016,NaN,4.629206,3.786349,4.524422,3.236886,NaN
9,4.117590,5.092622,4.957092,6.008443,4.463882,4.953263,5.484107,4.007013,3.435276,3.114530,3.113517,5.257124,5.100499,5.221177,5.403750,2.861648,5.060008,3.384264,3.780105,4.993054,NaN,4.623002,4.898054,6.120372,4.846679,4.189029,3.420054,5.600473,3.626286,4.258540,6.277342,NaN,NaN,NaN,NaN,3.494759,5.257100,6.164399,NaN,5.052332,4.685809,4.642659,2.669799,NaN


## Calculate the standarised monthly anomalies

After creating the dataframe that includes a multi-year monthly log discharge, we calculate the monthly anomalies using the mean and standard deviation

In [67]:
MONTHLY_ANOMALY = pd.DataFrame()
for n in YEAR_SELECTED:
    data = (LOG_DISCHARGE_MONTHLY_PIVOT[n] - DISCHARGE_MONTHLY_STATS['mean'])/DISCHARGE_MONTHLY_STATS['std']
    MONTHLY_ANOMALY[n] = data

In [68]:
MONTHLY_ANOMALY

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,...,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266,-0.810341
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,...,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152,-0.951652
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,...,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803,-1.550890
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,...,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546,-1.664257
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,...,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636,NaN
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,...,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390,NaN
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,...,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415,NaN
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,...,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212,NaN
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,...,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643,NaN


re-write the index from 1 to 12 (JAN to DEC)

In [69]:
MONTHLY_ANOMALY.index = np.arange(1, len(MONTHLY_ANOMALY) + 1)
HTML(MONTHLY_ANOMALY.to_html())

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,-1.217702,-0.332189,-0.151861,-0.391850,1.297849,-0.771544,-0.773628,-0.609809,1.599805,2.014703,-0.752051,NaN,0.168765,1.662446,0.220363,0.214728,2.427332,-0.405921,-0.069919,-0.275124,1.082403,-0.171486,NaN,0.908379,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266,-0.810341
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,0.695973,-0.834223,-0.451418,1.702978,NaN,0.080346,-1.114598,-0.735161,-0.535340,2.378751,-1.082684,NaN,-0.004572,0.958376,-0.386063,0.492906,-0.137558,-0.672549,-0.303736,0.213084,2.828374,-0.399055,NaN,-0.278387,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152,-0.951652
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,0.368636,-0.618578,-0.621814,-0.434609,0.184724,-0.028342,-1.316506,-0.671217,-0.821700,1.549766,-1.341889,1.498400,2.527140,0.214817,-0.607739,-0.278698,0.094414,2.332151,0.803340,0.482773,0.549333,-0.536844,NaN,NaN,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803,-1.550890
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,1.740413,-0.820711,-0.109710,0.941890,-0.084077,1.142900,0.569727,-1.152541,-0.223646,0.620392,NaN,-0.303012,2.078428,-0.689754,-0.127231,1.688282,-0.487829,1.956936,-0.653459,-0.651857,0.446257,-0.679586,-0.586552,NaN,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546,-1.664257
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,0.055118,-0.026225,0.897623,0.929453,1.142730,0.193611,-0.909332,-1.229191,-0.437847,-0.628618,2.064755,-0.136805,1.454932,0.670987,0.324343,1.216989,-0.986345,1.713232,-0.864665,-1.028210,-0.305526,-0.990349,-1.220725,NaN,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636,NaN
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,0.182126,0.362392,0.517625,0.271876,-0.628118,0.309844,-1.732159,-0.216433,-0.312289,0.386102,0.505539,1.243150,0.734981,1.084724,-0.109196,1.533202,1.317109,-0.217848,-0.893280,-0.582289,0.612672,NaN,-1.206965,-1.173551,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390,NaN
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,-1.619038,0.623227,0.285137,-0.455937,-0.508600,0.499323,-2.199121,-1.788752,0.929367,1.672127,1.241263,-0.321207,0.434812,0.506555,-0.236939,0.311917,0.214526,-1.384830,-0.367381,1.067286,1.057527,NaN,-0.782419,NaN,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415,NaN
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,-1.997518,0.865586,-0.708144,0.028734,0.605638,-1.407229,-2.770442,-0.163174,-0.543317,-0.011291,NaN,0.712015,0.528228,0.476646,-0.759065,-1.105045,0.331471,-0.278306,-1.034313,-0.365984,0.993314,NaN,NaN,NaN,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212,NaN
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,-1.517861,0.693688,0.532099,0.656601,0.844961,-1.777712,0.490324,-1.238533,-0.830146,0.421248,NaN,0.039467,0.323237,1.584296,0.270234,-0.408260,-1.201608,1.047919,-0.988840,-0.336547,1.746241,NaN,NaN,NaN,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643,NaN
10,0.640823,-0.748537,NaN,1.573028,0.291037,0.794642,0.913218,0.584085,-1.280330,-1.961382,0.017501,1.130842,0.270075,1.335728,0.181266,-0.460783,-0.214463,-0.144519,-1.455970,-1.266806,NaN,1.297236,-0.353800,-

In [70]:
MONTHLY_ANOMALY.index.name = 'Month'

In [71]:
MONTHLY_ANOMALY

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,...,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266,-0.810341
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,...,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152,-0.951652
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,...,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803,-1.550890
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,...,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546,-1.664257
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,...,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636,NaN
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,...,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390,NaN
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,...,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415,NaN
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,...,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212,NaN
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,...,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643,NaN


## Save dataframe of monthly  ANOMALY and DISCHARGE

In [72]:
ANOMALY_TS = pd.melt(MONTHLY_ANOMALY.reset_index(), id_vars='Month',var_name=['Year'],ignore_index = True)
ANOMALY_TS.columns = ['Month','Year','Anomaly_Qlog']

In [73]:
MONTHLY_ANOMALY

,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Month,,,,,,,,,,,,,,,,,,,,,
1,-2.123750,-0.308179,-1.551073,-0.796712,0.727208,-1.244110,-0.670261,-0.504405,0.818331,-1.280020,...,NaN,0.204800,0.814189,0.200969,0.324539,NaN,-0.385739,-0.230398,1.141266,-0.810341
2,0.316909,0.511895,0.235052,0.100800,2.454052,-1.259258,-0.370705,-0.773939,0.187623,-1.368724,...,NaN,-0.394508,-0.887051,0.577828,-0.659732,NaN,-0.742382,0.860497,-0.252152,-0.951652
3,1.120851,-0.362955,-0.375811,-0.951718,1.193772,0.744396,-0.898463,0.402005,1.742555,-0.994765,...,NaN,-0.677517,-1.096631,-0.545900,0.112274,-0.449993,-1.059530,0.710564,-0.389803,-1.550890
4,1.200881,-0.162872,-0.336847,-0.364767,-0.160803,0.583685,-0.569182,-0.573368,0.695763,-0.983002,...,NaN,NaN,2.118149,-0.969378,-0.979009,-1.319303,-1.188353,0.670950,-0.613546,-1.664257
5,1.373637,1.804841,0.930711,-0.734193,0.722645,0.782769,0.315669,-0.976373,-0.788636,-1.846345,...,NaN,-1.283716,0.049850,0.107734,-0.537679,0.053286,-1.607545,-0.087954,-0.178636,NaN
6,0.796236,-0.163576,1.175735,0.284660,1.159690,0.689027,0.508366,-1.715410,-1.236878,-2.084692,...,NaN,-1.814261,-0.301783,-0.850826,-0.520173,1.715769,0.585062,1.115231,-1.331390,NaN
7,0.669569,0.327620,0.605223,-0.332697,1.575762,0.619012,0.095685,-0.033314,-0.667932,-1.610597,...,NaN,-2.458694,1.464433,0.166312,0.883619,-0.018995,-0.531656,-0.129605,0.197415,NaN
8,0.585540,0.927440,0.726022,1.307074,-0.886873,0.710619,2.020854,-0.018238,-0.082788,-0.035092,...,NaN,0.983076,0.589616,1.700595,NaN,0.098847,-0.737193,-0.005090,-1.282212,NaN
9,-0.481964,0.523972,0.384146,1.468819,-0.124697,0.380196,0.927865,-0.596046,-1.185904,-1.516816,...,NaN,-1.124536,0.693663,1.629718,NaN,0.482405,0.104265,0.059747,-1.975643,NaN


In [74]:
DISCHARGE_MONTHLY['Anomaly_Qlog'] = ANOMALY_TS['Anomaly_Qlog'][0:len(DISCHARGE_MONTHLY['Discharge'])].values

In [75]:
DISCHARGE_MONTHLY

,Discharge,Year,Month,water_year,Q_to_log,Anomaly_Qlog
Fecha,,,,,,
1980-01-31,1.198613,1980,1,1980,0.181165,-2.123750
1980-02-29,28.386414,1980,2,1980,3.345911,0.316909
1980-03-31,86.782323,1980,3,1980,4.463403,1.120851
1980-04-30,172.187367,1980,4,1981,5.148583,1.200881
1980-05-31,275.560258,1980,5,1981,5.618806,1.373637
...,...,...,...,...,...,...
2022-12-31,5.969037,2022,12,2023,1.786586,-1.063870
2023-01-31,5.419419,2023,1,2023,1.689989,-0.810341
2023-02-28,5.557464,2023,2,2023,1.715142,-0.951652


### Save  MONTHLY_ANOMALY

In [76]:
MONTHLY_ANOMALY.to_csv('../data/santalucia_caudales_abr2023_MA.csv')

### Save  DISCHARGE_MONTHLY

In [77]:
DISCHARGE_MONTHLY.to_csv('../data/santalucia_caudales_abr2023_DA.csv')